In [10]:
import ollama
from wikipedia_tools import tools_available
from wiki_agent import WikipediaAgent

In [11]:
tools_available

[<function wikipedia_tools.save_results_to_path(content: str, file_name: str, file_directory: str = None) -> str>,
 <function wikipedia_tools.read_results_from_path(path: str = None) -> str>,
 <function wikipedia_tools.get_top_k_keywords(content: str, k: int) -> list[str]>,
 <function wikipedia_tools.search_wikipedia(query: str) -> str>]

In [12]:
file_save_format: str = """Question: {user question}\nTopic: {topic}\nSummary: {summary of results}""" # format we expect agent to follow when saving results
system_prompt = f"""You are an expert researcher agent that leverages wikipedia to answer questions. You are not to answer questions without using a tool or using information resulting from a tool. For example, if you are asked "What is the capital of France?", you should use the "search_wikipedia" tool to find the answer. If a user later asks, "What is the capital of France?", you may simply answer based off of your previous findings. In summary, query for new questions, for redundant questions, use your previous findings. 

Tools:
You have access to these tools
1. search_wikipedia(query: str)  
  - Search Wikipedia for a given query and return the summary text.
2. save_results_to_path(content: str, file_name: str, file_directory: str)  
  - Save text to a file at the specified directory.
3. get_top_k_keywords(content: str, k: int)  
  - Extract the k most frequent keywords from the content.
4. read_results_from_path(path: str)
  - reads the content of a file at a given path.


Output guidelines:
You must always respond in the following JSON format:
{{“intent”:…, “status”: ..., “function”: {{ “name”:…, “parameters”:...}}}}
- The "intent" field should be a short description of what you are trying to do.
- The "status" field should be one of the following: "in_progress", "done", or "request_human_approval".
  - "in_progress" means you are still working on the task and will need to call another tool.
  - "done" means you have completed the task and have all the information you need to answer the user's question.
  - "request_human_approval" means you need human approval before proceeding.
- The "function" field should contain the name of the tool being called and its parameters.
  - The parameters should be a dictionary containing the necessary arguments for the tool. For example, "parameters": {{"query": "What is the capital of France?"}}.

Do not include any other text outside of the JSON!
- Always respond with valid JSON using standard double quotes ".
- Do not use “ or ' for quotes.

File saving format:
For any requests that require saving the results the format should be as followed:
{file_save_format}.
- If the user does not specify a specific file name, you must name the file with the convention wiki_findings_i.txt"""

In [13]:
messages= [{'role': 'system', 'content': system_prompt},{'role': 'user', 'content': 'What is the status of the Sudanese Civil War in August 2025?'}]
response = ollama.chat(model="llama3.1", messages=messages)

In [14]:
response

ChatResponse(model='llama3.1', created_at='2025-09-05T02:57:13.404386924Z', done=True, done_reason='stop', total_duration=10081528585, load_duration=144121225, prompt_eval_count=544, prompt_eval_duration=206410059, eval_count=55, eval_duration=9723389049, message=Message(role='assistant', content='{"intent": "Find information on the Sudanese Civil War", \n"status": "in_progress", \n"function": {"name": "search_wikipedia", \n"parameters": {"query": "Sudanese Civil War status August 2025"}}}', thinking=None, images=None, tool_name=None, tool_calls=None))

In [15]:
llm_response = response.message.content

In [16]:
llm_response

'{"intent": "Find information on the Sudanese Civil War", \n"status": "in_progress", \n"function": {"name": "search_wikipedia", \n"parameters": {"query": "Sudanese Civil War status August 2025"}}}'

In [17]:
agent = WikipediaAgent(
  model_name="llama3.1",
  tools=tools_available,
  system_prompt=system_prompt,
)

In [18]:
agent.parse_agent_response(response=llm_response)

{'intent': 'Find information on the Sudanese Civil War',
 'status': 'in_progress',
 'function': {'name': 'search_wikipedia',
  'parameters': {'query': 'Sudanese Civil War status August 2025'}}}